In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
159,AFG,Asia,Afghanistan,2020-06-17,26310.0,783.0,491.0,13.0,675.857,20.114,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
260,ALB,Europe,Albania,2020-06-17,1672.0,82.0,37.0,1.0,580.999,28.494,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
425,DZA,Africa,Algeria,2020-06-17,11147.0,116.0,788.0,11.0,254.201,2.645,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-16,France,157755.0,0.0,29454.0,0.0,0.0,0.0
0,2020-06-16,China,84458.0,0.0,4650.0,0.0,0.0,0.0
0,2020-06-16,Italy,237752.0,0.0,34438.0,0.0,0.0,0.0
0,2020-06-16,Spain,244490.0,0.0,27146.0,0.0,0.0,0.0
0,2020-06-16,United States,2136724.0,0.0,116637.0,0.0,0.0,0.0
0,2020-06-16,World,8164154.0,0.0,440574.0,0.0,0.0,0.0
0,2020-06-16,United Kingdom,298718.0,0.0,41840.0,0.0,0.0,0.0
0,2020-06-16,Germany,187127.0,0.0,8816.0,0.0,0.0,0.0
0,2020-06-16,Iran,192593.0,0.0,9052.0,0.0,0.0,0.0
0,2020-06-16,Turkey,181274.0,0.0,4839.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-17,France,157755.0,157716.0,29454.0,29547.0,39.0,-93.0
0,2020-06-17,China,84458.0,84422.0,4650.0,4644.0,36.0,6.0
0,2020-06-17,Italy,237752.0,237500.0,34438.0,34405.0,252.0,33.0
0,2020-06-16,Spain,244490.0,244328.0,27146.0,27136.0,162.0,10.0
0,2020-06-17,United States,2136724.0,2137731.0,116637.0,116963.0,-1007.0,-326.0
0,2020-06-17,World,8164154.0,8142129.0,440574.0,443488.0,22025.0,-2914.0
0,2020-06-17,United Kingdom,298718.0,298136.0,41840.0,41969.0,582.0,-129.0
0,2020-06-17,Germany,187127.0,187184.0,8816.0,8830.0,-57.0,-14.0
0,2020-06-17,Iran,192593.0,192439.0,9052.0,9065.0,154.0,-13.0
0,2020-06-17,Turkey,181274.0,181298.0,4839.0,4842.0,-24.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")